# **Librerias necesarias**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# **Obtención de los datos**

In [2]:
dataset=pd.read_csv('../db/transacDC2017.csv',sep=';')

In [3]:
dataset.head()

,client_id,date,mcc,country_code,amount_sol,amount_usd,nb_transaction,client_age,client_gender,debit_type,merchant_id,merchant_name,merchant_type,merchant_geoid,merchant_departement,merchant_province,merchant_district,merchant_lon,merchant_lat
0,PJ1MmUtnJoo=,2016-06-01 20:22:13+02,5533,PE,230.0,70.919580,1,69.0,M,TD,52438230,IZA MOTORS PERU SAC,Venta de repuestos para vehiculos,150132,LIMA,LIMA,San Juan De Luriganc,-11.979603,-77.004344
1,AQhFrCiSF78=,2016-09-13 23:10:35+02,5533,PE,14.0,4.316844,1,43.0,F,TD,52438230,IZA MOTORS PERU SAC,Venta de repuestos para vehiculos,150132,LIMA,LIMA,San Juan De Luriganc,-11.979603,-77.004344
2,aBM/ALhZK9k=,2017-04-13 16:21:15+02,5533,PE,345.0,106.379370,1,24.0,M,TD,52438230,IZA MOTORS PERU SAC,Venta de repuestos para vehiculos,150132,LIMA,LIMA,San Juan De Luriganc,-11.979603,-77.004344
3,8f3H6sKptnU=,2017-04-04 18:17:22+02,5533,PE,460.0,141.839160,1,70.0,M,TD,52438230,IZA MOTORS PERU SAC,Venta de repuestos para vehiculos,150132,LIMA,LIMA,San Juan De Luriganc,-11.979603,-77.004344
4,WoNB5LNBqHw=,2016-11-27 13:26:51+01,5533,PE,12.0,3.700152,1,38.0,M,TD,52438230,IZA MOTORS PERU SAC,Venta de repuestos para vehiculos,150132,LIMA,LIMA,San Juan De Luriganc,-11.979603,-77.004344


# **Pre procesamiento**

### **1) Imputacion de la edad de los clientes**

**Intento de imputación de datos en la columna client_age basandose en la media de cada uno de los tipos de negocio(merchant_type). Falla debido a que en el tipo de negocio que faltan datos no existe ni un solo dato, es decir que todos son NAN, es por eso que la media tambien sale NAN**

In [4]:
#Imputacion de la edad faltante haciendo uso de la media de cada uno de los tipos de negocio(merchant_type)
pd.options.mode.chained_assignment = None 
def imput_age():
    merchant_type=dataset.merchant_type.unique() #Obtenemos todos los tipos de negocio
    merchant_mean={}#Diccionario para almacenar las medias por cada negocio. Forma {tipo_negocio:media}
    for merchant in merchant_type:
        merchant_mean[merchant]=dataset.client_age[dataset.merchant_type == merchant].mean() #Almacenamos la media
        if True in (dataset[dataset.merchant_type == merchant].isnull().any(axis=1)): #Comprobamos si el tipo de negocio tiene valores vacios para imputar
            dataset.client_age[dataset.merchant_type == merchant]=dataset.client_age[dataset.merchant_type == merchant].fillna(merchant_mean[merchant])
    return merchant_mean
    

In [5]:
imput_age()

{'Venta de repuestos para vehiculos': 42.40708705357143,
 'Discotecas, peñas, bares y pubs': 30.977995986238533,
 'Tiendas por departamentos': 40.35379734085415,
 'Restaurants': 35.35444654985203,
 'Pago de Impuestos': 43.24818401937046,
 'Equipos de Telecomunicaciones': 38.13540766349755,
 'Farmacias, droguerias y perfumerias': 38.08521027738719,
 'Laboratorios medicos y dentales': 37.76720351390922,
 'Casinos y juegos de azar': 40.787186000564496,
 'Cafeterias, snacks, servicio express': 34.679869133443354,
 'Grifos, servicio': 38.37651483493523,
 'Venta de Pinturas, Empapelados,Vidrierias y Esp.': 41.493261455525605,
 'Peluquerias': 36.405902102845026,
 'Hoteles': 34.310172570390556,
 'Opticos': 38.982142857142854,
 'Talleres de reparacion general para vehiculos': 42.34108527131783,
 'Zapaterias': 36.02783361682388,
 'Tiendas de ventas especializadas al menudeo': 33.30014641288433,
 'Dentistas, ortodoncistas': 33.95185185185185,
 'Alquiler equipo y mobiliario domestico o indust.': 4

### **2) Transformación del tipo de datos**

In [6]:
cols_to_transform=['mcc','merchant_id','merchant_geoid']
for col in cols_to_transform:
    dataset[col]=dataset[col].astype(str)

### **3) Dropeo de columnas innecesarias**

**Dropearemos las columnas que tienen valores únicos**

In [7]:
dataset.shape

(300000, 19)

In [8]:
cols_to_drop=['country_code','merchant_departement','merchant_province']
dataset=dataset.drop(cols_to_drop,axis=1)

In [9]:
dataset.shape

(300000, 16)

### **4) Splitting date**

In [10]:
def splittingDate(data):
    año=[datetime.strptime(date.split('+')[0], "%Y-%m-%d %H:%M:%S").year for date in data.date]
    mes=[datetime.strptime(date.split('+')[0], "%Y-%m-%d %H:%M:%S").month for date in data.date]
    dia=[datetime.strptime(date.split('+')[0], "%Y-%m-%d %H:%M:%S").day for date in data.date] 
    hora=[datetime.strptime(date.split('+')[0], "%Y-%m-%d %H:%M:%S").hour for date in data.date] 
    minuto=[datetime.strptime(date.split('+')[0], "%Y-%m-%d %H:%M:%S").minute for date in data.date] 
    segundo=[datetime.strptime(date.split('+')[0], "%Y-%m-%d %H:%M:%S").second for date in data.date] 
    
    data['Year']=año
    data['Month']=mes
    data['Day']=dia
    data['Hour']=hora
    data['Minute']=minuto
    data['Second']=segundo
    #eliminamos la columna que se dividió
    #data.drop('Last Updated',axis=1,inplace=True)

In [11]:
splittingDate(dataset)

In [12]:
dataset.head()

,client_id,date,mcc,amount_sol,amount_usd,nb_transaction,client_age,client_gender,debit_type,merchant_id,...,merchant_geoid,merchant_district,merchant_lon,merchant_lat,Year,Month,Day,Hour,Minute,Second
0,PJ1MmUtnJoo=,2016-06-01 20:22:13+02,5533,230.0,70.919580,1,69.0,M,TD,52438230,...,150132,San Juan De Luriganc,-11.979603,-77.004344,2016,6,1,20,22,13
1,AQhFrCiSF78=,2016-09-13 23:10:35+02,5533,14.0,4.316844,1,43.0,F,TD,52438230,...,150132,San Juan De Luriganc,-11.979603,-77.004344,2016,9,13,23,10,35
2,aBM/ALhZK9k=,2017-04-13 16:21:15+02,5533,345.0,106.379370,1,24.0,M,TD,52438230,...,150132,San Juan De Luriganc,-11.979603,-77.004344,2017,4,13,16,21,15
3,8f3H6sKptnU=,2017-04-04 18:17:22+02,5533,460.0,141.839160,1,70.0,M,TD,52438230,...,150132,San Juan De Luriganc,-11.979603,-77.004344,2017,4,4,18,17,22
4,WoNB5LNBqHw=,2016-11-27 13:26:51+01,5533,12.0,3.700152,1,38.0,M,TD,52438230,...,150132,San Juan De Luriganc,-11.979603,-77.004344,2016,11,27,13,26,51
